# Tic Tac Toe Training Notebook.

## Consists of
- Driver/main function
- Agent
- Actions
- Environment
- Reward cycle

## Major components
- State
- Actions
- Rewards
- Q Learning
```
Q(S,A) = (1-∂)*Q(S,A) + (∂ * µ * maxaQ(S',a))
```
    - S = Current state
    - A = Current action
    - S' = State after current action
    - ∂ = Learning rate
    - µ = Discount factor
    - maxa(S',a) = Highest Q value of any move in next state S'

In [ ]:
import numpy as np
import random
import copy

## Tic Tac Toe Class

In [ ]:
class TicTacToe():
  def __init__(self, size):

    self.size = size
    self.board = np.zeros([self.size, self.size], dtype = int)
    self.p1 = 1
    self.p2 = -1
    self.turn = 1

  def print_board(self):
    print("-------")
    for i in range(self.size):
        print("|", end="")
        for spot in self.board[i]:
            if spot != 0:
                if spot == 1:
                  print('X', end="")
                  print("|", end="")
                elif spot == -1:
                  print('O', end="")
                  print("|", end="")
            else:
                print(" |", end="")
        print("\n-------")

  def reset_board(self):
    self.board = np.zeros([self.size,self.size])
    self.turn = 1

  #check winner
  def done(self):

    # Check horizontal locations for win
    for row in range(self.size):
      if np.sum(self.board[row,:]) == self.p1 * self.size:
        return self.p1
      elif np.sum(self.board[row,:]) == self.p2 * self.size:
        return self.p2

    # Check vertical locations for win
    for col in range(self.size):
      if np.sum(self.board[:,col]) == self.p1 * self.size:
        return self.p1
      elif np.sum(self.board[:,col]) == self.p2 * self.size:
        return self.p2

    # Check positively sloped diagonals
    if np.trace(np.fliplr(self.board)) == self.p1 * self.size:
      return self.p1
    if np.trace(np.fliplr(self.board)) == self.p2 * self.size:
      return self.p2

     # Check negatively sloped diagonals
    if np.trace(self.board) == self.p1 * self.size:
      return self.p1
    if np.trace(self.board) == self.p2 * self.size:
      return self.p2

    if len(self.actions()) == 0:
      return 0

    return None

  #return availble actions
  def actions(self):

    states = list()
    flat = self.board.flatten()
    for i in range(len(flat)):
        if flat[i] == 0:
          states.append(i)

    return states

  #make a move on the board
  def step(self, action):

    #get the row and columns based on a value from a flattened version of board
    row = action // self.size
    col = action % self.size

    if self.turn == 1:
      self.board[row, col] = self.p1
    elif self.turn == -1:
      self.board[row, col] = self.p2

    if self.turn == 1:
      self.turn = -1
    elif self.turn == -1:
      self.turn = 1

  def reward(self):

    if self.done() == 1:
      return 10, self.p1
    elif self.done() == -1:
      return 10, self.p2
    elif self.done() == 0:
      return 0, None

  def hash(self):
    board_hash = str(self.board.flatten())

    return board_hash




## Player Class

In [ ]:
class Player():
  def __init__(self, name, exp_rate = 0.5):
    self.name = name
    self.states = []
    self.lr = 0.5
    self.discount = 0.7
    self.exp_rate = exp_rate
    self.exp_decay = 0.99
    self.exp_min = 0.1
    self.states_value = {}   #("       ": [000000000]) initial state

  def updateExp(self):
    if self.exp_rate > self.exp_min:
      self.exp_rate *= self.exp_decay

  def updateStateValues(self, reward):
    #give reward to final move
    finalMove = self.states[len(self.states)-1]
    self.states_value[finalMove[0]][finalMove[1]] = reward

    for i in range(len(self.states)-2, -1, -1):
      self.states_value[self.states[i][0]][self.states[i][1]] = ((1-self.lr)*self.states_value[self.states[i][0]][self.states[i][1]] + self.lr*self.discount*np.amax(self.states_value[self.states[i+1][0]]))

    self.states = []

  #check if board is in our dictionary
  def updateDict(self, board_hash, size):

    if board_hash not in self.states_value:
      self.states_value[board_hash] = np.zeros((size*size), dtype=float)
      return True

    return False

  #if moves is not in list of possible moves set it to a low value so algoritm never picks it
  def updateDictImpossible(self, board_hash, moves):

      for i in range(len(self.states_value[board_hash])):
        if i not in moves:
          self.states_value[board_hash][i] = -20

  def updateStates(self, board_hash, move):
    self.states.append((board_hash, move))


  def chooseAction(self, positions, current_board):
    if np.random.uniform(0,1) <= self.exp_rate:

      #random action
      choice = np.random.choice(len(positions))
      action = positions[choice]
    else:
      value_max = -99
      for p in positions:
        hash = current_board.hash()
        moveScore = self.states_value[hash][p]
        if moveScore > value_max:
                value_max = moveScore
                action = p


    return action

  # Get Q value of state.
  # Add states once explored.
  def get_q_values(self, board_hash):

    if board_hash in self.states_value:
        qvals = self.states_value[board_hash]
    else:
        qvals = np.full(9, self.q_init_val)
        self.states_value[board_hash] = qvals

    return qvals

# Training and Game play

## 3 x 3

In [ ]:
enviroment = TicTacToe(3)
p1 = Player("X3x3")
p2 = Player("O3x3")
reward = 10
episodeCount = 2000

In [ ]:
for episodes in range(episodeCount):
  done = False

  enviroment.reset_board()

  while not done:
    actions = enviroment.actions()
    boardHash = enviroment.hash()

    #check whose turn it is
    if enviroment.turn == 1:
      #add board hash to the states
      added = p1.updateDict(boardHash, enviroment.size)
      #if board hash was never in dict update for values that can't be picked
      if added:
        p1.updateDictImpossible(boardHash,actions)
      #choose a move
      move = p1.chooseAction(actions,enviroment)
      #add move to list of moves
      p1.updateStates(boardHash, move)

    elif enviroment.turn == -1:
      #add board hash to the states
      added = p2.updateDict(boardHash, enviroment.size)
      #if board hash was never in dict update for values that can't be picked
      if added:
        p2.updateDictImpossible(boardHash,actions)
      #choose a move
      move = p2.chooseAction(actions, enviroment)
      #add move to list of moves
      p2.updateStates(boardHash, move)

    #apply move to board
    enviroment.step(move)
    #check if game is done and apply rewards
    finished = enviroment.done()

    #p1 wins
    if finished == 1:
      #update dict for p1, positive reward
      p1.updateStateValues(reward)
      #update dict for p2, negative reward
      p2.updateStateValues(-reward)

      done = True

    #p2 wins
    elif finished == -1:
      #update dict for p2, positive reward
      p2.updateStateValues(reward)
      #update dict for p1, negative reward
      p1.updateStateValues(-reward)

      done = True

    #draw
    elif finished == 0:
      #update dict for p1, neutral reward
      p1.updateStateValues(1)
      #update dict for p2, neutral reward
      p2.updateStateValues(1)

      done = True


In [ ]:
enviroment.reset_board()
done = False
while not done:
    actions = enviroment.actions()
    boardHash = enviroment.hash()

    #check whose turn it is
    if enviroment.turn == 1:
      #add board hash to the states
      added = p1.updateDict(boardHash, enviroment.size)
      #if board hash was never in dict update for values that can't be picked
      if added:
        p1.updateDictImpossible(boardHash,actions)
      #choose a move
      move = p1.chooseAction(actions,enviroment)
      #add move to list of moves
      p1.updateStates(boardHash, move)

    elif enviroment.turn == -1:
        enviroment.print_board()
        print('Please choose one of the possible board moves.')
        print(actions)
        move = int(input())

    #apply move to board
    enviroment.step(move)
    #check if game is done and apply rewards
    finished = enviroment.done()

    #p1 wins
    if finished == 1:
      #update dict for p1, positive reward
      p1.updateStateValues(reward)
      print("Computer wins")

      done = True

    #p2 wins
    elif finished == -1:

      #update dict for p1, negative reward
      p1.updateStateValues(-reward)
      print("you win")
      done = True

    #draw
    elif finished == 0:
      #update dict for p1, neutral reward
      p1.updateStateValues(1)
      print("tie game")
      done = True

enviroment.print_board()

-------
| | | |
-------
|X| | |
-------
| | | |
-------
Please choose one of the possible board moves.
[0, 1, 2, 4, 5, 6, 7, 8]
0
-------
|O| | |
-------
|X|X| |
-------
| | | |
-------
Please choose one of the possible board moves.
[1, 2, 5, 6, 7, 8]
5
-------
|O| |X|
-------
|X|X|O|
-------
| | | |
-------
Please choose one of the possible board moves.
[1, 6, 7, 8]
6
-------
|O|X|X|
-------
|X|X|O|
-------
|O| | |
-------
Please choose one of the possible board moves.
[7, 8]
8
Computer wins
-------
|O|X|X|
-------
|X|X|O|
-------
|O|X|O|
-------


## 4 x 4

In [ ]:
enviroment2 = TicTacToe(4)
play1 = Player("X4x4")
play2 = Player("O4x4")
reward = 10
episodeCount = 2000

In [ ]:
for episodes in range(episodeCount):
  done = False

  enviroment2.reset_board()

  while not done:
    actions = enviroment2.actions()
    boardHash = enviroment2.hash()
    #check whose turn it is
    if enviroment2.turn == 1:
      #add board hash to the states
      added = play1.updateDict(boardHash, enviroment2.size)
      #if board hash was never in dict update for values that can't be picked
      if added:
        play1.updateDictImpossible(boardHash,actions)
      #choose a move
      move = play1.chooseAction(actions,enviroment2)
      #add move to list of moves
      play1.updateStates(boardHash, move)

    elif enviroment2.turn == -1:
      #add board hash to the states
      added = play2.updateDict(boardHash, enviroment2.size)
      #if board hash was never in dict update for values that can't be picked
      if added:
        play2.updateDictImpossible(boardHash,actions)
      #choose a move
      move = play2.chooseAction(actions, enviroment2)
      #add move to list of moves
      play2.updateStates(boardHash, move)

    #apply move to board
    enviroment2.step(move)
    #check if game is done and apply rewards
    finished = enviroment2.done()

    #p1 wins
    if finished == 1:
      #update dict for p1, positive reward
      play1.updateStateValues(reward)
      #update dict for p2, negative reward
      play2.updateStateValues(-reward)

      done = True

    #p2 wins
    elif finished == -1:
      #update dict for p2, positive reward
      play2.updateStateValues(reward)
      #update dict for p1, negative reward
      play1.updateStateValues(-reward)

      done = True

    #draw
    elif finished == 0:
      #update dict for p1, neutral reward
      play1.updateStateValues(1)
      #update dict for p2, neutral reward
      play2.updateStateValues(1)

      done = True

In [ ]:
enviroment2.reset_board()
done = False
while not done:
    actions = enviroment2.actions()
    boardHash = enviroment2.hash()
    #check whose turn it is
    if enviroment2.turn == 1:
      #add board hash to the states
      added = play1.updateDict(boardHash, enviroment2.size)
      #if board hash was never in dict update for values that can't be picked
      if added:
        play1.updateDictImpossible(boardHash,actions)
      #choose a move
      move = play1.chooseAction(actions,enviroment2)
      #add move to list of moves
      play1.updateStates(boardHash, move)

    elif enviroment2.turn == -1:
        enviroment2.print_board()
        print('Please choose one of the possible board moves.')
        print(actions)
        move = int(input())
    #apply move to board
    enviroment2.step(move)
    #check if game is done and apply rewards
    finished = enviroment2.done()

    #p1 wins
    if finished == 1:
      #update dict for p1, positive reward
      play1.updateStateValues(reward)
      print("Computer wins")
      done = True

    #p2 wins
    elif finished == -1:
      #update dict for p1, neg reward
      play1.updateStateValues(reward)
      print("you win")
      done = True

    #draw
    elif finished == 0:
      #update dict for p1, neutral reward
      play1.updateStateValues(1)
      print("tie game")
      done = True

enviroment2.print_board()

-------
| |X| | |
-------
| | | | |
-------
| | | | |
-------
| | | | |
-------
Please choose one of the possible board moves.
[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
0
-------
|O|X| | |
-------
| | |X| |
-------
| | | | |
-------
| | | | |
-------
Please choose one of the possible board moves.
[2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15]
2
-------
|O|X|O| |
-------
| |X|X| |
-------
| | | | |
-------
| | | | |
-------
Please choose one of the possible board moves.
[3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15]
7
-------
|O|X|O| |
-------
| |X|X|O|
-------
| | | | |
-------
| | | |X|
-------
Please choose one of the possible board moves.
[3, 4, 8, 9, 10, 11, 12, 13, 14]
11
-------
|O|X|O|X|
-------
| |X|X|O|
-------
| | | |O|
-------
| | | |X|
-------
Please choose one of the possible board moves.
[4, 8, 9, 10, 12, 13, 14]
8
-------
|O|X|O|X|
-------
|X|X|X|O|
-------
|O| | |O|
-------
| | | |X|
-------
Please choose one of the possible board moves.
[9, 10, 12, 13, 14]
9
-------


# Authors
- Luis Abeyta
- Wesley Johnson
- Noah Warren
- Michael Youssef